# Verifying Seebuoy data for NDBC on Station 44150

In [1]:
import sys

!{sys.executable} -m pip install seebuoy

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os

path = os.getcwd()
buoyNum = 44065

In [3]:
import requests

url = f"https://www.ndbc.noaa.gov/data/realtime2/{buoyNum}.txt"
r = requests.get(url, allow_redirects=True)
open(f"{path}/{buoyNum}.txt", 'wb').write(r.content)

/Users/gabekrishnadasan/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


613165

In [4]:
from seebuoy import NDBC
import pandas as pd
import numpy as np

ndbc = NDBC()

# Information on NDBC's ~1800 buoys and gliders
wave_df = ndbc.stations()

# list all available data for all buoys
df_data = ndbc.available_data()

# Get info on La Jave Bank (42.500N 64.02W) 
# SEEBUOY DATA
see_buoy = ndbc.get_data(str(buoyNum))
see_buoy = see_buoy[::-1]
see_buoy

,wind_direction,wind_speed,wind_gust,wave_height,dominant_period,average_period,mean_wave_direction,pressure,air_temp,water_temp,dewpoint,visibility,pressure_tendency,tide
date,,,,,,,,,,,,,,
2024-02-13 22:30:00,NaN,9.0,11.0,NaN,NaN,NaN,NaN,1004.4,NaN,5.8,NaN,NaN,NaN,NaN
2024-02-13 22:20:00,NaN,9.0,11.0,1.1,8.0,4.3,108.0,NaN,NaN,5.8,NaN,NaN,NaN,NaN
2024-02-13 22:10:00,NaN,9.0,11.0,1.1,NaN,4.3,108.0,1004.1,NaN,5.8,NaN,NaN,NaN,NaN
2024-02-13 22:00:00,NaN,10.0,12.0,NaN,NaN,NaN,NaN,1003.8,NaN,5.8,NaN,NaN,3.9,NaN
2024-02-13 21:50:00,NaN,10.0,13.0,1.2,8.0,4.4,109.0,1003.6,NaN,5.8,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-30 00:40:00,NaN,5.0,6.0,0.8,NaN,5.4,140.0,1004.1,NaN,9.5,NaN,NaN,NaN,NaN
2023-12-30 00:30:00,NaN,5.0,7.0,NaN,NaN,NaN,NaN,1004.2,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-30 00:20:00,NaN,4.0,6.0,0.8,9.0,5.4,124.0,1004.1,NaN,9.6,NaN,NaN,NaN,NaN


In [5]:
# NDBC TXT
# Define the width of each column based on the spacing in your data
column_widths = [4, 3, 3, 3, 3, 5, 5, 5, 6, 6, 5, 5, 7, 7, 5, 5, 6, 4, 6]

# Read the fixed-width file into a Pandas DataFrame
ndbc = pd.read_fwf(f"{buoyNum}.txt", widths=column_widths, header=[0])
ndbc = ndbc.drop(labels=0, axis=0)
ndbc

,#YY,MM,DD,hh,mm,WDIR,WSPD,GST,WVHT,DPD,APD,MWD,PRES,ATMP,WTMP,DEWP,VIS,PTDY,TIDE
1,2024,02,13,22,30,MM,9.0,11.0,MM,MM,MM,MM,1004.4,MM,5.8,MM,MM,MM,MM
2,2024,02,13,22,20,MM,9.0,11.0,1.1,8,4.3,108,MM,MM,5.8,MM,MM,MM,MM
3,2024,02,13,22,10,MM,9.0,11.0,1.1,MM,4.3,108,1004.1,MM,5.8,MM,MM,MM,MM
4,2024,02,13,22,00,MM,10.0,12.0,MM,MM,MM,MM,1003.8,MM,5.8,MM,MM,+3.9,MM
5,2024,02,13,21,50,MM,10.0,13.0,1.2,8,4.4,109,1003.6,MM,5.8,MM,MM,MM,MM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6517,2023,12,30,00,40,MM,5.0,6.0,0.8,MM,5.4,140,1004.1,MM,9.5,MM,MM,MM,MM
6518,2023,12,30,00,30,MM,5.0,7.0,MM,MM,MM,MM,1004.2,MM,MM,MM,MM,MM,MM
6519,2023,12,30,00,20,MM,4.0,6.0,0.8,9,5.4,124,1004.1,MM,9.6,MM,MM,MM,MM
6520,2023,12,30,00,10,MM,5.0,6.0,0.8,MM,5.4,124,1004.2,MM,9.6,MM,MM,MM,MM


In [6]:
# consolidate columns into one date column
ndbc['date'] = ndbc.apply(lambda row: f"{int(row['#YY'])}-{int(row['MM']):02d}-{int(row['DD']):02d} {int(row['hh']):02d}:{int(row['mm']):02d}:00", axis=1)

#ndbc = ndbc.drop(['#YY', 'MM', 'DD', 'hh', 'mm'], axis=1)

# replace columns with see_buoy column names
ndbc.rename(columns={'WDIR': 'wind_direction', 'WSPD': 'wind_speed', 'GST':'wind_gust', 'WVHT':'wave_height', 'DPD':'dominant_period',
                        'APD':'average_period', 'MWD':'mean_wave_direction', 'PRES':'pressure', 'ATMP':'air_temp', 'WTMP':'water_temp', 'DEWP':'dewpoint',
                        'VIS':'visibility', 'PTDY':'pressure_tendency', 'TIDE':'tide'}, inplace=True)

# replace MM with NaN
ndbc.replace('MM', np.nan, inplace=True)
ndbc

,#YY,MM,DD,hh,mm,wind_direction,wind_speed,wind_gust,wave_height,dominant_period,average_period,mean_wave_direction,pressure,air_temp,water_temp,dewpoint,visibility,pressure_tendency,tide,date
1,2024,02,13,22,30,NaN,9.0,11.0,NaN,NaN,NaN,NaN,1004.4,NaN,5.8,NaN,NaN,NaN,NaN,2024-02-13 22:30:00
2,2024,02,13,22,20,NaN,9.0,11.0,1.1,8,4.3,108,NaN,NaN,5.8,NaN,NaN,NaN,NaN,2024-02-13 22:20:00
3,2024,02,13,22,10,NaN,9.0,11.0,1.1,NaN,4.3,108,1004.1,NaN,5.8,NaN,NaN,NaN,NaN,2024-02-13 22:10:00
4,2024,02,13,22,00,NaN,10.0,12.0,NaN,NaN,NaN,NaN,1003.8,NaN,5.8,NaN,NaN,+3.9,NaN,2024-02-13 22:00:00
5,2024,02,13,21,50,NaN,10.0,13.0,1.2,8,4.4,109,1003.6,NaN,5.8,NaN,NaN,NaN,NaN,2024-02-13 21:50:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6517,2023,12,30,00,40,NaN,5.0,6.0,0.8,NaN,5.4,140,1004.1,NaN,9.5,NaN,NaN,NaN,NaN,2023-12-30 00:40:00
6518,2023,12,30,00,30,NaN,5.0,7.0,NaN,NaN,NaN,NaN,1004.2,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-30 00:30:00
6519,2023,12,30,00,20,NaN,4.0,6.0,0.8,9,5.4,124,1004.1,NaN,9.6,NaN,NaN,NaN,NaN,2023-12-30 00:20:00
6520,2023,12,30,00,10,NaN,5.0,6.0,0.8,NaN,5.4,124,1004.2,NaN,9.6,NaN,NaN,NaN,NaN,2023-12-30 00:10:00


In [7]:
ndbc.columns

Index(['#YY', 'MM', 'DD', 'hh', 'mm', 'wind_direction', 'wind_speed',
       'wind_gust', 'wave_height', 'dominant_period', 'average_period',
       'mean_wave_direction', 'pressure', 'air_temp', 'water_temp', 'dewpoint',
       'visibility', 'pressure_tendency', 'tide', 'date'],
      dtype='object')

In [8]:
see_buoy.dropna(axis=1, how='all', inplace=True)
ndbc.dropna(axis=1, how='all', inplace=True)

In [9]:
see_buoy

,wind_speed,wind_gust,wave_height,dominant_period,average_period,mean_wave_direction,pressure,water_temp,pressure_tendency
date,,,,,,,,,
2024-02-13 22:30:00,9.0,11.0,NaN,NaN,NaN,NaN,1004.4,5.8,NaN
2024-02-13 22:20:00,9.0,11.0,1.1,8.0,4.3,108.0,NaN,5.8,NaN
2024-02-13 22:10:00,9.0,11.0,1.1,NaN,4.3,108.0,1004.1,5.8,NaN
2024-02-13 22:00:00,10.0,12.0,NaN,NaN,NaN,NaN,1003.8,5.8,3.9
2024-02-13 21:50:00,10.0,13.0,1.2,8.0,4.4,109.0,1003.6,5.8,NaN
...,...,...,...,...,...,...,...,...,...
2023-12-30 00:40:00,5.0,6.0,0.8,NaN,5.4,140.0,1004.1,9.5,NaN
2023-12-30 00:30:00,5.0,7.0,NaN,NaN,NaN,NaN,1004.2,NaN,NaN
2023-12-30 00:20:00,4.0,6.0,0.8,9.0,5.4,124.0,1004.1,9.6,NaN


In [10]:
ndbc

,#YY,MM,DD,hh,mm,wind_speed,wind_gust,wave_height,dominant_period,average_period,mean_wave_direction,pressure,water_temp,pressure_tendency,tide,date
1,2024,02,13,22,30,9.0,11.0,NaN,NaN,NaN,NaN,1004.4,5.8,NaN,NaN,2024-02-13 22:30:00
2,2024,02,13,22,20,9.0,11.0,1.1,8,4.3,108,NaN,5.8,NaN,NaN,2024-02-13 22:20:00
3,2024,02,13,22,10,9.0,11.0,1.1,NaN,4.3,108,1004.1,5.8,NaN,NaN,2024-02-13 22:10:00
4,2024,02,13,22,00,10.0,12.0,NaN,NaN,NaN,NaN,1003.8,5.8,+3.9,NaN,2024-02-13 22:00:00
5,2024,02,13,21,50,10.0,13.0,1.2,8,4.4,109,1003.6,5.8,NaN,NaN,2024-02-13 21:50:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6517,2023,12,30,00,40,5.0,6.0,0.8,NaN,5.4,140,1004.1,9.5,NaN,NaN,2023-12-30 00:40:00
6518,2023,12,30,00,30,5.0,7.0,NaN,NaN,NaN,NaN,1004.2,NaN,NaN,NaN,2023-12-30 00:30:00
6519,2023,12,30,00,20,4.0,6.0,0.8,9,5.4,124,1004.1,9.6,NaN,NaN,2023-12-30 00:20:00
6520,2023,12,30,00,10,5.0,6.0,0.8,NaN,5.4,124,1004.2,9.6,NaN,NaN,2023-12-30 00:10:00


In [11]:
cols = see_buoy.columns
cols

Index(['wind_speed', 'wind_gust', 'wave_height', 'dominant_period',
       'average_period', 'mean_wave_direction', 'pressure', 'water_temp',
       'pressure_tendency'],
      dtype='object')

In [12]:
for col in cols:
    ndbc[col] = ndbc[col].astype(float)
    see_buoy.dropna(subset=[col], inplace=True)
    ndbc.dropna(subset=[col], inplace=True)
    see_buoyList = []
    ndbcList = []

    for value in see_buoy[col]:
        see_buoyList.append(value)

    for val in ndbc[col]:
        ndbcList.append(val)

    for i in range(len(see_buoyList)):
        if see_buoyList[i] != ndbcList[i]:
            print(col)
            print(i)
            print("see_buoy ", see_buoyList[i])
            print("ndbc ", ndbcList[i])
            print()